In [ ]:
## In Class 2021-09-15

In [17]:
## Q1 Load the data file to you S3 bucket. Using the pandas library, read the csv data file and
## create a data-frame called fish.

import pandas as pd
import boto3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

## Defining the S3 bucket
s3 = boto3.resource('s3')
bucket_name = 'bonnieh-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'Fish.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
fish = pd.read_csv(file_content_stream)
fish.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [18]:
## Q2 Using the Length1, Lenght2, Length3, Height, and Width as the predictor variables, and
## Weight is the target variable, split the data into train (80%) and test (20%).

## Defining the variables

X = fish[['Length1', 'Length2', 'Length3', 'Height', 'Width']]
Y = fish['Weight']

## splitting the data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20)

In [19]:
## Q3 Using the train dataset, perform LASSO as a variable selector.

## running cross-validation and estimating lambda (alpha)

lasso_cv = LassoCV(normalize = True, cv = 5, max_iter=10000).fit(X_train, Y_train)

## increase number of iterations if get error asking for more

## extracting the optimal lambda

lasso_alpha = lasso_cv.alpha_
lasso_alpha

## Building the lasso model
lasso_md = Lasso(alpha = lasso_alpha, normalize = True, max_iter = 10000).fit(X_train, Y_train)
lasso_md.coef_

## results show to drop variables 2 and 3--length2 and length3

array([20.49717444,  0.        ,  0.        , 13.28271839, 58.31275155])

In [20]:
## Dropping the variables with zeroes
## X_train = X_train.drop(columns = ['Length2', 'Length3'], axis = 1)
X_test = X_test.drop(columns = ['Length2', 'Length3'], axis = 1)
X_train = X_train.drop(columns = ['Length2', 'Length3'], axis = 1)
X_train

,Length1,Height,Width
146,10.0,1.9720,1.1600
72,7.5,2.1120,1.4080
131,34.8,6.2884,4.0198
52,24.0,8.8768,4.4968
115,34.6,10.5717,6.3666
...,...,...,...
24,31.9,16.2405,5.5890
143,56.0,9.6000,6.1440
87,20.0,5.6400,3.5250
35,12.9,4.1472,2.2680


In [21]:
## Q4 Using the variables from LASSO regression, normalize the inputs variables of the train and
## test datasets using the L2 normalization. That is, for each input variable subtract the mean of that
## variable, then divide by the L2-norm of that variable.

def l2_normalization(X):
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X-x_mean) /l2

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [22]:
## Q5  Using the train dataset, build a linear regression model. After that, use this model to predict
## on the test dataset. Report the MSE of this model.

md1 = LinearRegression().fit(X_train, Y_train)

## predict on test data set

md1_pred = md1.predict(X_test)
md1_pred

## computing MSE

mse1 = np.mean(np.power(md1_pred - Y_test, 2))
mse1

134216.21149577605

In [23]:
## Q6 Using the train dataset, build a ridge regression model. After that, use this model to predict
## on the test dataset. Report the MSE of this model.

ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1,1,10, 100], cv = 5).fit(X_train, Y_train)
ridge_alpha = ridge_cv.alpha_

## building the ridge model
ridge_md = Ridge(alpha = ridge_alpha).fit(X_train, Y_train)

## predicting on the test dataset
md2_pred = ridge_md.predict(X_test)

## Computing the mse
mse2 = np.mean(np.power(md2_pred - Y_test, 2))
mse2

133840.03408265926

In [25]:
## Q7 Using the results from parts (5) and (6), what model would you use to predict fish weight?
## Explain.

## calculate difference between mse1 and mse1

mse1-mse2

## the difference is not very large for this split of the data. Model 1 (the linear regression model)
## is slightly better than model 2 (the ridge regression model).  Different data splits definitely 
## affect the results for this data set

376.1774131167913